# 100x10 Hearthstone Arena Challenge
### The rules:
In 10 arena runs in a row get 100 wins. You cannot pick a hero you played before unless all three choices are heroes you've played before.

The popular Hearthstone streamer, Hafu, is the closest person to completing this challenge at 96 wins. I'll use statistics from her stream for my simulation.

In [100]:
#class name, mean, standard deviation
probs <- array(c("Paladin",8.33,2.5,"Rogue",8.15,3.22,"Mage",7.86,2.59,
                 "Druid",7.84,3.02,"Priest",7.26,2.65,"Warlock",6.82,3.15,
                 "Warrior",6.77,2.83,"Hunter",6.74,2.81,"Shaman",6.64,2.61),c(3,9))

# stats from here https://docs.google.com/spreadsheets/d/1jck4WkEvaKWE0Iw9RrttLHoFO3KEMG6In6XS1okzmx4/edit#gid=1077822399

I'm going to assume that the distribution is normal. It very likely could not be normal, but I don't konw.

In [101]:
#The code to generate the 10 games for one attempt at the challenge
sim <- function(fun){
  classes <- 1:9
  playedClasses <- NULL
  run <- array(0,c(2,11)) # an array to put the results of the run into
  choices <- sample(9,3) # gets 3 classes to choose from
  chosen <- fun(choices, run) # selects the 1 class from the 3 chosen
  playedClasses[1] <- chosen # adds this class to a list that keeps track of the classes played
  run[1,1] = probs[1,chosen] # puts the class name into the run array
  # A random number generated from a normal distribution using the classes mean and standard diviation.
  wins = floor(rnorm(1,mean = as.double(probs[2,chosen]),sd = as.double(probs[3,chosen])))
  if(wins > 12) wins = 12 # lowers the number to 12 if it goes above it
  run[2,1] <- wins # puts the number of wins in the results array
  for(i in 2:10){ # the second to tenth runs
    choices <- sample(9,3) # gets 3 classes to choose from
    # gets a list of classes that are from the 3 to choose from and classes already played
    temp <- intersect(choices, playedClasses) 
    # if all three arent the same then it uses the ones from the original list that havent been played
    if(length(temp) != 3) choices = setdiff(choices,temp) 
    # chooses 1 class from the ones that are allowed
    chosen <- fun(choices, run)
    run[1,i] = probs[1,chosen]
    wins <- floor(rnorm(1,mean = as.double(probs[2,chosen]),sd = as.double(probs[3,chosen])))
    if(wins > 12) wins = 12
    run[2,i] <- wins
    playedClasses[i] <- chosen
  }
  # hack fix because streamer will only choose to start run on 12 wins.
  run[2,1] = "12"
  # add total wins to results array
  run[1,11] <- "total"
  run[2,11] <- sum(as.double(run[2,]))
  run
}

I'll find the first thirty successful runs and see if I can find some trends

In [102]:
# strategy functions i'll use to determine the selection

selectBadFirst <- function(choices, run){
    if(length(choices)==1){
        return(choices[1])
    }

    #if first game, pick worst class
    if(run[2,1] == 0){
        return(max(choices))
    }else {
        return(min(choices))
    }
}

selectBestAlways <- function(choices, run){

    return(min(choices))
}

In [103]:
# find first n successful runs with selecting the best class always
classlevels.allRuns <- 0
totalAttempts <- 0
n = 1000
for(j in 1:n){
    win = TRUE
    run <- NULL
    count = 0
    
    while(win){
      run <- sim(selectBestAlways)
      count = count + 1
      if(as.numeric(run[2,11]) >= 100){
        print(run)
        classLevelsTotal <- NULL
        classLevels <- NULL
        for(i in 1:10){
            if(i >1)
            classLevelsTotal[i] <- classLevelsTotal[i-1] + (which(run[1,i]==probs[1,]))
            else
            classLevelsTotal[i] <- which(run[1,i]==probs[1,])
            classLevels[i] <- which(run[1,i]==probs[1,])
        }
        print(classLevels)
        print(classLevelsTotal)
        print(count)
        totalAttempts <- totalAttempts + count
        classlevels.allRuns <- classlevels.allRuns + classLevelsTotal[10]
        win = FALSE
      }
    }

}


     [,1]    [,2]      [,3]      [,4]    [,5]     [,6]      [,7]   [,8]     
[1,] "Rogue" "Paladin" "Warlock" "Druid" "Priest" "Paladin" "Mage" "Warrior"
[2,] "12"    "10"      "11"      "11"    "10"     "9"       "11"   "10"     
     [,9]      [,10]     [,11]  
[1,] "Paladin" "Paladin" "total"
[2,] "7"       "10"      "101"  
 [1] 2 1 6 4 5 1 3 7 1 1
 [1]  2  3  9 13 18 19 22 29 30 31
[1] 719
     [,1]     [,2]    [,3]    [,4]      [,5]   [,6]      [,7]      [,8]   
[1,] "Priest" "Druid" "Rogue" "Paladin" "Mage" "Warlock" "Warrior" "Druid"
[2,] "12"     "7"     "8"     "9"       "12"   "7"       "11"      "12"   
     [,9]      [,10]   [,11]  
[1,] "Paladin" "Rogue" "total"
[2,] "10"      "12"    "100"  
 [1] 5 4 2 1 3 6 7 4 1 2
 [1]  5  9 11 12 15 21 28 32 33 35
[1] 808
     [,1]    [,2]      [,3]      [,4]     [,5]   [,6]    [,7]    [,8]     
[1,] "Druid" "Paladin" "Warlock" "Priest" "Mage" "Rogue" "Rogue" "Paladin"
[2,] "12"    "12"      "12"      "9"      "10"   "12"    "10"    "

In [104]:
c(classlevels.allRuns, classlevels.allRuns/n)
c(totalAttempts, totalAttempts/n)

[1] 41086.000    41.086

[1] 1237251.000    1237.251

In [105]:
# find first n successful runs with selecting a bad class first
classlevels.allRuns <- 0
totalAttempts <- 0
for(j in 1:n){
    win = TRUE
    run <- NULL
    count = 0
    
    while(win){
      run <- sim(selectBadFirst)
      count = count + 1
      if(as.numeric(run[2,11]) >= 100){
        print(run)
        classLevelsTotal <- NULL
        classLevels <- NULL
        for(i in 1:10){
            if(i >1)
            classLevelsTotal[i] <- classLevelsTotal[i-1] + (which(run[1,i]==probs[1,]))
            else
            classLevelsTotal[i] <- which(run[1,i]==probs[1,])
            classLevels[i] <- which(run[1,i]==probs[1,])
        }
        print(classLevels)
        print(classLevelsTotal)
        print(count)
        totalAttempts <- totalAttempts + count
        classlevels.allRuns <- classlevels.allRuns + classLevelsTotal[10]
        win = FALSE
      }
    }

}


     [,1]     [,2]      [,3]    [,4]    [,5]   [,6]     [,7]      [,8]     
[1,] "Shaman" "Paladin" "Druid" "Rogue" "Mage" "Priest" "Warlock" "Paladin"
[2,] "12"     "12"      "12"    "11"    "9"    "6"      "9"       "12"     
     [,9]      [,10]     [,11]  
[1,] "Paladin" "Warrior" "total"
[2,] "9"       "8"       "100"  
 [1] 9 1 4 2 3 5 6 1 1 7
 [1]  9 10 14 16 19 24 30 31 32 39
[1] 17
     [,1]     [,2]      [,3]   [,4]    [,5]     [,6]    [,7]    [,8]   
[1,] "Priest" "Paladin" "Mage" "Druid" "Hunter" "Rogue" "Rogue" "Rogue"
[2,] "12"     "11"      "11"   "6"     "12"     "12"    "10"    "11"   
     [,9]      [,10]    [,11]  
[1,] "Warlock" "Shaman" "total"
[2,] "9"       "7"      "101"  
 [1] 5 1 3 4 8 2 2 2 6 9
 [1]  5  6  9 13 21 23 25 27 33 42
[1] 104
     [,1]      [,2]    [,3]    [,4]      [,5]   [,6]    [,7]      [,8]     
[1,] "Warrior" "Druid" "Rogue" "Warlock" "Mage" "Rogue" "Paladin" "Paladin"
[2,] "12"      "10"    "12"    "12"      "9"    "11"    "12"      "6"     

In [106]:
c(classlevels.allRuns, classlevels.allRuns/n)
c(totalAttempts, totalAttempts/n)

[1] 42018.000    42.018

[1] 846400.0    846.4